# STS Benchmark Datasets

### Preparation

Setup all required libraries

In [1]:
import logging
import re
import sys

import numpy as np
import pandas as pd

from gensim.models.keyedvectors import KeyedVectors, FastTextKeyedVectors

from fse.models import Average, SIF, uSIF
from fse import IndexedList

from re import sub

from scipy.stats import pearsonr

from nltk import word_tokenize

logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

Next, we require the sentences from the STS benchmark dataset.

In [2]:
file= "../fse/eval/sts-test.csv"
similarities, sent_a, sent_b = [], [], []
with open(file, "r") as f:
    for l in f:
        line = l.rstrip().split("\t")
        similarities.append(float(line[4]))
        sent_a.append(line[5])
        sent_b.append(line[6])
similarities = np.array(similarities)
assert len(similarities) == len(sent_a) == len(sent_b)
task_length = len(similarities)

for i, obj in enumerate(zip(similarities, sent_a, sent_b)):
    print(f"{i}\tSim: {obj[0].round(3):.1f}\t{obj[1]:40s}\t{obj[2]:40s}\t")
    if i == 4:
        break

0	Sim: 2.5	A girl is styling her hair.             	A girl is brushing her hair.            	
1	Sim: 3.6	A group of men play soccer on the beach.	A group of boys are playing soccer on the beach.	
2	Sim: 5.0	One woman is measuring another woman's ankle.	A woman measures another woman's ankle. 	
3	Sim: 4.2	A man is cutting up a cucumber.         	A man is slicing a cucumber.            	
4	Sim: 1.5	A man is playing a harp.                	A man is playing a keyboard.            	


Each of these sentence requires some preparation (i.e. tokenization) to be used in the core input formats.
To reproduce the results from the uSIF paper this part is taken from https://github.com/kawine/usif/blob/master/usif.py

In [3]:
not_punc = re.compile('.*[A-Za-z0-9].*')

def prep_token(token):
    t = token.lower().strip("';.:()").strip('"')
    t = 'not' if t == "n't" else t
    return re.split(r'[-]', t)

def prep_sentence(sentence):
    tokens = []
    for token in word_tokenize(sentence):
        if not_punc.match(token):
            tokens = tokens + prep_token(token)
    return tokens

Next we define the IndexedList object. The IndexedList extends the previously constructed sent_a and sent_b list together. We additionally provide a custom function "prep_sentence" which performs all the prepocessing for a single sentence

In [4]:
sentences = IndexedList(sent_a, sent_b, split=False, split_func=prep_sentence)

The IndexedList returns the core object required for fse to train a sentence embedding: An IndexedSentence object. This object constists of words (a list of strings) and its corresponding index. The latter is important if multiple cores access the input queue simultaneously. Thus it must be always provided. The index represents the row in the matrix where it can later be found.

In [5]:
sentences[0]

IndexedSentence(words=['a', 'girl', 'is', 'styling', 'her', 'hair'], index=0)

Note, that IndexedList does not convert the sentences inplace but only on calling the __getitem__ method in order to turn the sentence into an IndexedSentence. You can access the original sentence using

In [6]:
sentences.items[0]

'A girl is styling her hair.'

### Loading the models

It is required for us to load the models as BaseKeyedVectors or as an BaseWordEmbeddingsModel. For this notebook, I already converted the models to a BaseKeyedVectors instance and saved the corresponding instance on my external harddrive. You have to replicate these steps yourself, because getting all the files can be a bit difficult, as the total filesize is around 15 GB.

In [7]:
path_to_models = "/Volumes/Ext_HDD/Models/Static/"
models, results = {}, {}

The following code performs a disk-to-ram training. Passing a path to __wv_mapfile_path__ will store the corresponding word vectors (wv) as a numpy memmap array. This is required, because loading all vectors into ram would would take up a lot of storage unecessary. The wv.vectors file will be replace by its memmap representation, which is why the next models do not require the wv_mapfile_path argument, as they access the same memmap object.

The lang_freq="en" induces the frequencies of the words according to the wordfreq package. This functionality allows you to work with pre-trained embeddings which don't come with frequency information. The method overwrites the counts in the glove.wv.vocab class, so that all further models also benefit from this induction.

In [8]:
glove = KeyedVectors.load(path_to_models+"glove.840B.300d.model")

print(f"Before memmap {sys.getsizeof(glove.vectors)}")

models[f"CBOW-Glove"] = Average(glove, wv_mapfile_path="data/glove", lang_freq="en")

print(f"After memmap {sys.getsizeof(glove.vectors)}")

models[f"SIF-Glove"] = SIF(glove, components=15)
models[f"uSIF-Glove"] = uSIF(glove,length=11)

2019-08-25 17:15:26,493 : MainThread : INFO : loading Word2VecKeyedVectors object from /Volumes/Ext_HDD/Models/Static/glove.840B.300d.model
2019-08-25 17:15:31,256 : MainThread : INFO : loading vectors from /Volumes/Ext_HDD/Models/Static/glove.840B.300d.model.vectors.npy with mmap=None
2019-08-25 17:15:35,998 : MainThread : INFO : setting ignored attribute vectors_norm to None
2019-08-25 17:15:35,999 : MainThread : INFO : loaded /Volumes/Ext_HDD/Models/Static/glove.840B.300d.model
2019-08-25 17:15:36,000 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/notebooks/data/glove_wv.vectors
2019-08-25 17:15:36,188 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


Before memmap 2635050112
After memmap 136


In [9]:
# Do all the vectors contain the same content?
(models[f"SIF-Glove"].wv.vectors == models[f"uSIF-Glove"].wv.vectors).all()

True

In [10]:
w2v = KeyedVectors.load(path_to_models+"google_news.model")

models[f"CBOW-W2V"] = Average(w2v, wv_mapfile_path="data/w2v", lang_freq="en")
models[f"SIF-W2V"] = SIF(w2v, components=10)
models[f"uSIF-W2V"] = uSIF(w2v, length=11)

2019-08-25 17:15:47,418 : MainThread : INFO : loading Word2VecKeyedVectors object from /Volumes/Ext_HDD/Models/Static/google_news.model
2019-08-25 17:15:54,700 : MainThread : INFO : loading vectors from /Volumes/Ext_HDD/Models/Static/google_news.model.vectors.npy with mmap=None
2019-08-25 17:16:01,682 : MainThread : INFO : setting ignored attribute vectors_norm to None
2019-08-25 17:16:01,683 : MainThread : INFO : loaded /Volumes/Ext_HDD/Models/Static/google_news.model
2019-08-25 17:16:01,684 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/notebooks/data/w2v_wv.vectors
2019-08-25 17:16:02,197 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


In [11]:
ft = FastTextKeyedVectors.load(path_to_models+"ft_crawl_300d_2m.model")
models[f"CBOW-FT"] = Average(ft, wv_mapfile_path="data/ft", lang_freq="en")
models[f"SIF-FT"] = SIF(ft, components=10)
models[f"uSIF-FT"] = uSIF(ft, length=11)

2019-08-25 17:16:04,775 : MainThread : INFO : loading FastTextKeyedVectors object from /Volumes/Ext_HDD/Models/Static/ft_crawl_300d_2m.model
2019-08-25 17:16:09,678 : MainThread : INFO : loading vectors from /Volumes/Ext_HDD/Models/Static/ft_crawl_300d_2m.model.vectors.npy with mmap=None
2019-08-25 17:16:14,004 : MainThread : INFO : loading vectors_vocab from /Volumes/Ext_HDD/Models/Static/ft_crawl_300d_2m.model.vectors_vocab.npy with mmap=None
2019-08-25 17:16:18,589 : MainThread : INFO : loading vectors_ngrams from /Volumes/Ext_HDD/Models/Static/ft_crawl_300d_2m.model.vectors_ngrams.npy with mmap=None
2019-08-25 17:16:22,946 : MainThread : INFO : setting ignored attribute vectors_norm to None
2019-08-25 17:16:22,955 : MainThread : INFO : setting ignored attribute vectors_vocab_norm to None
2019-08-25 17:16:22,957 : MainThread : INFO : setting ignored attribute vectors_ngrams_norm to None
2019-08-25 17:16:22,958 : MainThread : INFO : setting ignored attribute buckets_word to None
2019

In [12]:
paranmt = KeyedVectors.load(path_to_models+"paranmt.model")

models[f"CBOW-Paranmt"] = Average(paranmt, lang_freq="en")
models[f"SIF-Paranmt"] = SIF(paranmt, components=10)
models[f"uSIF-Paranmt"] = uSIF(paranmt, length=11)

2019-08-25 17:16:27,546 : MainThread : INFO : loading Word2VecKeyedVectors object from /Volumes/Ext_HDD/Models/Static/paranmt.model
2019-08-25 17:16:27,882 : MainThread : INFO : loading vectors from /Volumes/Ext_HDD/Models/Static/paranmt.model.vectors.npy with mmap=None
2019-08-25 17:16:28,053 : MainThread : INFO : setting ignored attribute vectors_norm to None
2019-08-25 17:16:28,054 : MainThread : INFO : loaded /Volumes/Ext_HDD/Models/Static/paranmt.model
2019-08-25 17:16:28,056 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


In [13]:
paragram = KeyedVectors.load(path_to_models+"paragram_sl999_czeng.model")

models[f"CBOW-Paragram"] = Average(paragram, lang_freq="en")
models[f"SIF-Paragram"] = SIF(paragram, components=10)
models[f"uSIF-Paragram"] = uSIF(paragram, length=11)

2019-08-25 17:16:28,152 : MainThread : INFO : loading Word2VecKeyedVectors object from /Volumes/Ext_HDD/Models/Static/paragram_sl999_czeng.model
2019-08-25 17:16:28,295 : MainThread : INFO : loading vectors from /Volumes/Ext_HDD/Models/Static/paragram_sl999_czeng.model.vectors.npy with mmap=None
2019-08-25 17:16:28,466 : MainThread : INFO : setting ignored attribute vectors_norm to None
2019-08-25 17:16:28,467 : MainThread : INFO : loaded /Volumes/Ext_HDD/Models/Static/paragram_sl999_czeng.model
2019-08-25 17:16:28,470 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


## Computation of the results for the STS benchmark

We are finally able to compute the STS benchmark values for all models.

In [14]:
# This function is used to compute the similarities between two sentences.
# Task length is the length of the sts dataset.
def compute_similarities(task_length, model):
    sims = []
    for i, j in zip(range(task_length), range(task_length, 2*task_length)):
        sims.append(model.sv.similarity(i,j))
    return sims

In [15]:
for k, m in models.items():
    m_type  = k.split("-")[0]
    emb_type = k.split("-")[1]
    m.train(sentences)
    r = pearsonr(similarities, compute_similarities(task_length, m))[0].round(4) * 100
    results[f"{m_type}-{emb_type}"] = r
    print(k, f"{r:2.2f}")

2019-08-25 17:16:28,553 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-25 17:16:28,934 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2019-08-25 17:16:30,263 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 2195875 vocabulary: 2524 MB (2 GB)
2019-08-25 17:16:30,264 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2019-08-25 17:16:30,315 : MainThread : INFO : begin training
2019-08-25 17:16:30,801 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-25 17:16:30,802 : MainThread : INFO : training on 2758 effective sentences with 27351 effective words took 0s with 5668 sentences/s
2019-08-25 17:16:30,827 : MainThread : INFO : scanning all indexed sentences and their word counts


CBOW-Glove 40.41


2019-08-25 17:16:31,225 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2019-08-25 17:16:31,990 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 2195875 vocabulary: 2524 MB (2 GB)
2019-08-25 17:16:31,991 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2019-08-25 17:16:32,000 : MainThread : INFO : pre-computing SIF weights for 2195875 words
2019-08-25 17:16:33,529 : MainThread : INFO : begin training
2019-08-25 17:16:33,894 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-25 17:16:33,931 : MainThread : INFO : computing 15 principal components took 0s
2019-08-25 17:16:33,933 : MainThread : INFO : removing 15 principal components took 0s
2019-08-25 17:16:33,934 : MainThread : INFO : training on 2758 effective sentences with 27351 effective words took 0s with 7527 sentences/s
2019-08-25 17:16:33,962 : MainThread : INFO : scanning all indexe

SIF-Glove 71.95


2019-08-25 17:16:34,344 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2019-08-25 17:16:35,102 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 2195875 vocabulary: 2524 MB (2 GB)
2019-08-25 17:16:35,103 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2019-08-25 17:16:35,112 : MainThread : INFO : pre-computing uSIF weights for 2195875 words
2019-08-25 17:16:41,638 : MainThread : INFO : begin training
2019-08-25 17:16:42,009 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-25 17:16:42,024 : MainThread : INFO : computing 5 principal components took 0s
2019-08-25 17:16:42,026 : MainThread : INFO : removing 5 principal components took 0s
2019-08-25 17:16:42,026 : MainThread : INFO : training on 2758 effective sentences with 27351 effective words took 0s with 7406 sentences/s
2019-08-25 17:16:42,056 : MainThread : INFO : scanning all indexed

...

CBOW-Paragram 50.38


2019-08-25 17:17:17,218 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2019-08-25 17:17:17,249 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 77224 vocabulary: 91 MB (0 GB)
2019-08-25 17:17:17,250 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2019-08-25 17:17:17,261 : MainThread : INFO : pre-computing SIF weights for 77224 words
2019-08-25 17:17:17,324 : MainThread : INFO : begin training
2019-08-25 17:17:17,820 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-25 17:17:17,841 : MainThread : INFO : computing 10 principal components took 0s
2019-08-25 17:17:17,843 : MainThread : INFO : removing 10 principal components took 0s
2019-08-25 17:17:17,845 : MainThread : INFO : training on 2758 effective sentences with 27441 effective words took 0s with 5545 sentences/s
2019-08-25 17:17:17,883 : MainThread : INFO : scanning all indexed sent

SIF-Paragram 73.86


2019-08-25 17:17:18,329 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2019-08-25 17:17:18,360 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 77224 vocabulary: 91 MB (0 GB)
2019-08-25 17:17:18,361 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2019-08-25 17:17:18,368 : MainThread : INFO : pre-computing uSIF weights for 77224 words
2019-08-25 17:17:18,623 : MainThread : INFO : begin training
2019-08-25 17:17:19,032 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-25 17:17:19,047 : MainThread : INFO : computing 5 principal components took 0s
2019-08-25 17:17:19,049 : MainThread : INFO : removing 5 principal components took 0s
2019-08-25 17:17:19,050 : MainThread : INFO : training on 2758 effective sentences with 27441 effective words took 0s with 6724 sentences/s


uSIF-Paragram 73.64


In [16]:
pd.DataFrame.from_dict(results, orient="index", columns=["Pearson"])

,Pearson
CBOW-Glove,40.41
SIF-Glove,71.95
uSIF-Glove,67.16
CBOW-W2V,61.54
SIF-W2V,71.12
uSIF-W2V,66.99
CBOW-FT,48.49
SIF-FT,73.38
uSIF-FT,69.40
CBOW-Paranmt,79.85


If you closely study the values above you will find that:
- SIF-Glove is almost equivalent to the values reported in http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark
- CBOW-Paranmt is a little better than ParaNMT Word Avg. in https://www.aclweb.org/anthology/W18-3012
- uSIF-Paranmt is a little worse than ParaNMT+UP in https://www.aclweb.org/anthology/W18-3012
- uSIF-Paragram is a little worse than PSL+UP in https://www.aclweb.org/anthology/W18-3012

However, I guess those differences might arise due to differences in preprocessing. Too bad we didn't hit 80. If you have any ideas why those values don't match exactly, feel free to contact me anytime.